In [57]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import functional as F

In [58]:
class RBM(nn.Module):
    def __init__(self, n_vis, n_hid):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_hid, n_vis))
        self.v_bias = nn.Parameter(torch.randn(n_vis))
        self.h_bias = nn.Parameter(torch.randn(n_hid))
        
        # update the weight matrices to match the input dimensions
        nn.init.xavier_uniform_(self.W)
        self.W.data = self.W.data.T

    def forward(self, v):
        h_prob = torch.sigmoid(F.linear(v, self.W, self.h_bias))
        h = torch.bernoulli(h_prob)
        v_prob = torch.sigmoid(F.linear(h, self.W.t(), self.v_bias))
        v = torch.bernoulli(v_prob)
        return v, v_prob, h, h_prob

In [59]:
class DBN(nn.Module):
    def __init__(self, layers, n_hids=None):
        super(DBN, self).__init__()

        if n_hids is None:
            n_hids = [64] * (len(layers) - 1)
            print(f"No. of hidden layers not specified. Setting default no. of hidden units to 500 for all layers.")
        elif len(n_hids) != len(layers) - 1:
            raise ValueError(f"Invalid number of hidden layers: expected {len(layers)-1}, got {len(n_hids)}")

        self.rbms = nn.ModuleList()
        for i in range(1, len(layers)):
            n_vis = layers[i-1]
            rbm = RBM(n_vis, n_hids[i-1])
            self.rbms.append(rbm)

        self.fc = nn.Linear(n_hids[-1], 10)

    def forward(self, x):
        h = x
        for rbm in self.rbms:
            h = rbm.forward(h)

        h = h.view(h.size(0), -1)
        y = self.fc(h)

        return y


In [60]:
# Load Fashion-MNIST dataset
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

In [61]:
# Set hyperparameters
learning_rate = 0.01
num_epochs = 10
batch_size = 64
layers = [784, 500, 250, 100, 10]



In [64]:
# Initialize DBN model
n_hids = [784, 500, 250, 100, 10]   
if len(n_hids) != len(layers) - 1:
    n_hids = [500] * (len(layers) - 1)
    print(f"Invalid number of hidden layers: expected {len(layers)-1}, setting default number of hidden units to {n_hids}")
model = DBN(layers, n_hids=n_hids)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


ValueError: Invalid number of hidden layers: expected 4, got 5

In [63]:
# Train the DBN model
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 784)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x784 and 500x784)

In [ ]:
# Test the DBN model
with torch.no_grad():
    correct = 0
    total = 0
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    for images, labels in test_loader:
        images = images.view(-1, 784)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the DBN model on the {} test images: {} %'.format(total, 100 * correct / total))
